### POC train and test the model with JoeyNMT

In [1]:
import os
from pathlib import Path

In [13]:
#chnage the config file and the model directory
path_to_config_file= "../JoeyYamlFiles/L2_0308_300.yaml"
path_to_model_dir= "../models/L2_0308_300/"

In [15]:
import yaml
import torch

# Die Konfiguration wird geladen
with open(os.path.join(os.pardir,"JoeyYamlFiles/L2_0308_300.yaml"), 'r') as file:
  config = yaml.safe_load(file)

# if GPU available, train on GPU
if config["training"]["use_cuda"] == True and torch.cuda.is_available() == True:
  print("--- Training on GPU ---")
  !python -m joeynmt train {path_to_config_file}

# no GPU available, switch to CPU
else:
  config["training"]["use_cuda"] = False
  with open(os.path.join(os.pardir,"JoeyYamlFiles/L2_0308_300.yaml"), 'w') as file:
    yaml.dump(config, file)
  print("--- Training on CPU ---")
  !python -m joeynmt train {path_to_config_file}

--- Training on CPU ---
Traceback (most recent call last):
  File "/usr/lib64/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib64/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/bernadeta/python-venvs/jupyternb-python3.6/lib64/python3.6/site-packages/joeynmt/__main__.py", line 41, in <module>
    main()
  File "/home/bernadeta/python-venvs/jupyternb-python3.6/lib64/python3.6/site-packages/joeynmt/__main__.py", line 29, in main
    train(cfg_file=args.config_path)
  File "/home/bernadeta/python-venvs/jupyternb-python3.6/lib64/python3.6/site-packages/joeynmt/training.py", line 713, in train
    overwrite=cfg["training"].get("overwrite", False))
  File "/home/bernadeta/python-venvs/jupyternb-python3.6/lib64/python3.6/site-packages/joeynmt/helpers.py", line 42, in make_model_dir
    "Model directory exists and overwriting is disabled.")
FileExistsError: Model directory exists and overwriting is disabled.


In [ ]:
#start predicting with the trained model, predictions are being saved in the models directory
!python3 -m joeynmt test {path_to_config_file} --output_path {path_to_model_dir}
